<a href="https://colab.research.google.com/github/sujala111/Coursera_Capstone/blob/main/Coursera_Capstone_Week3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Data Science Capstone Project**

# Importing Libraries


In [56]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Hello Capstone Project Course!')

Libraries imported.
Hello Capstone Project Course!


In [52]:
!pip install beautifulsoup4

# Loading the data

In [58]:
url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(url,'lxml')

In [59]:
table_post = soup.find('table') # finding table in html file
fields = table_post.find_all('td') #finding td

postcode = [] # to append postal codes
borough = [] # to append borough
neighbourhood = [] # to append neighbourhood

for i in range(0, len(fields), 3):
    postcode.append(fields[i].text.strip()) 
    borough.append(fields[i+1].text.strip())
    neighbourhood.append(fields[i+2].text.strip())
        
toronto_data = pd.DataFrame(data=[postcode, borough, neighbourhood]).transpose()
toronto_data.columns = ['Postcode', 'Borough', 'Neighbourhood']
toronto_data.head(12)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


## Deleting rows that contains Borough with value "not assigned"

In [60]:
toronto_data['Borough'].replace('Not assigned', np.nan, inplace=True)
toronto_data.dropna(subset=['Borough'],inplace=True)

In [74]:
toronto_data.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


## Grouping Neighbours

In [72]:
toronto_data_combine.columns = ['Postcode', 'Borough', 'Neighbourhood']

toronto_data_combine = toronto_data.groupby(['Postcode','Borough'])['Neighbourhood'].apply(','.join).reset_index()
toronto_data_combine.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [73]:
toronto_data_combine.shape 

(103, 3)

## Reading Geospatial data

In [76]:
geo = pd.read_csv('https://cocl.us/Geospatial_data')
geo.columns=['Postcode','Borough','Neighbourhood']

In [77]:
geo.head()

,Postcode,Borough,Neighbourhood
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


# Merging two dataframes

In [80]:
toronto = pd.merge(toronto_data_combine, geo, on=['Postcode'])
toronto.columns=['Postcode','Borough','Neighbourhood','Latitude','Longitude']

In [82]:
toronto

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [87]:
toronto.shape

(103, 5)